In [192]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

In [167]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 29/12/22 - Horário: 11:38:16.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:20.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:42:20.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:35.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:48.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:44:27.  Portanto não foi baixado novamente.
vrapeelo

In [168]:
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']

In [169]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)

In [170]:
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)

In [171]:
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)


cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

input_rapeel = pd.merge(mercado,ambiental_datas,on=ide_data_cols,how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=ide_data_cols,how="inner")
input_rapeel = input_rapeel.loc[input_rapeel.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]


In [172]:
cols = ["IdeUsinaOutorga","DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]

In [173]:
definicoes = pd.merge(input_rapeel,ug_rapeel,on=ide_data_cols,how='inner')
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]

In [174]:
cols = ["DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","IdeUsinaOutorga","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]

monitoramentodados = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]

In [175]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao
rename_cols = {'DatPrevistaComercial':'DatPrevistaComercial'}
definicoes_monitoramento = pd.merge(monitoramentodados,definicoes,on='IdeUsinaOutorga',how="left").rename(columns=rename_cols).drop_duplicates()
definicoes_monitoramento = definicoes_monitoramento.loc[definicoes_monitoramento.IdcUsinaMonitorada == "Sim"]

definicoes_monitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoes_monitoramento.loc[definicoes_monitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]
definicoes_monitoramento.loc[definicoes_monitoramento.CondicaoAmbiental.isna(),'CondicaoAmbiental'] = 'Sem LP'
definicoes_monitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)


definicoes_monitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSitContratoCCD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCCT == nsa) &
    definicoes_monitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoes_monitoramento.loc[
(    ((definicoes_monitoramento.NomSitContratoCCD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCCT == valido)) &
    definicoes_monitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSitContratoCCD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCCT == vencido) |
    (definicoes_monitoramento.NomSitContratoCCD == nass) |
    (definicoes_monitoramento.NomSitContratoCCT == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoes_monitoramento[['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoes_monitoramento['CondicaoUso'] = "Não informado"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCUST == vencido) |
    (definicoes_monitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoes_monitoramento.NomSitContratoCUST == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoUso'] = "Sem Uso"

display(definicoes_monitoramento[['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoes_monitoramento["PPA"] = "Ambos"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoes_monitoramento[['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
67442,Não se Aplica,Não Assinado,2022-12-01 14:33:25.137,Sem Conexão
76474,Não se Aplica,Não Assinado,2022-12-02 10:40:19.547,Sem Conexão
15599,Não se Aplica,Válido,2022-12-01 16:08:06.183,OK
34049,Não se Aplica,Não Assinado,2022-12-04 19:34:47.213,Sem Conexão
81829,Válido,Não se Aplica,2022-12-05 16:59:33.210,OK
20257,Não se Aplica,Não Assinado,2022-12-05 18:40:10.517,Sem Conexão
42725,Não Assinado,Não se Aplica,2022-12-03 12:50:15.877,Sem Conexão
40487,Válido,Não se Aplica,2022-12-01 18:24:07.953,OK
485,Não se Aplica,Não se Aplica,2022-12-05 15:59:52.300,Verificar
44699,Não se Aplica,Válido,2022-12-04 09:57:44.477,OK


,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
13522,Não se Aplica,Não Assinado,Sem Conexão
40298,Não se Aplica,Não Assinado,Sem Conexão
2620,Não se Aplica,Válido,OK
127,Vencido,Não se Aplica,Sem Conexão
61857,Não se Aplica,Válido,OK
43692,Não se Aplica,Válido,OK
27742,Não se Aplica,Válido,OK
1253,Não se Aplica,Válido,OK
78083,Não se Aplica,Não se Aplica,Verificar
34326,Não se Aplica,Válido,OK


,DscComercializacaoEnergia,DatConclusaoACL,PPA
61931,Fora do ACR,NaT,Nenhum
43389,Fora do ACR,NaT,Nenhum
44920,Fora do ACR,NaT,Nenhum
2600,ACR,2021-12-20,Ambos
55755,ACR,NaT,ACR
20199,Fora do ACR,NaT,Nenhum
8469,Fora do ACR,NaT,Nenhum
9101,ACR,NaT,ACR
37861,ACR,NaT,ACR
25269,Fora do ACR,NaT,Nenhum


In [176]:
monitoramentodados[monitoramentodados.IdeUsinaOutorga == 273]

,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra,IdcObraParalisada,IdcUsinaMonitorada,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento,IdeUsinaOutorga,SigTipoGeracao,DscComercializacaoEnergia,DatInicioObraRealizado,DatMontagemRealizado
0,2021-03-15,NaT,NaT,NaT,NaT,Não,Sim,2022-08-08,2021-10-14,NaT,2021-12-08,2022-03-15,2022-02-15,Em andamento,Não,NaT,2022-12-21 12:25:43.883,273,PCH,ACR,2021-03-15,2022-02-15


In [177]:
definicoes_monitoramento[definicoes_monitoramento.IdeUsinaOutorga == 273].drop_duplicates()[['IdeUsinaOutorga','DthEnvio']]

,IdeUsinaOutorga,DthEnvio
0,273,2022-10-09 21:04:17.743
1,273,2022-10-09 21:04:17.743


In [178]:
# Apenas mudança de nome
tabela_criterio = definicoes_monitoramento

In [179]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabela_criterio['criterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabela_criterio['dscriterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabela_criterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
68552,Não Iniciada,Nenhum,Sem LP,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
2712,Em andamento,Nenhum,LI,OK,0,Usina em Obras em andamento
21852,Em andamento,ACL,LI,OK,0,Usina em Obras em andamento
1226,Em andamento,Nenhum,LI,OK,0,Usina em Obras em andamento
2498,Em andamento,ACR,LO,Sem Uso,0,Usina em Obras em andamento
8821,Em andamento,ACR,LO,OK,0,Usina em Obras em andamento
35447,Não Iniciada,ACL,LI,OK,1,Sem obras e PPA Ok
68545,Não Iniciada,Nenhum,Sem LP,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
35550,Em andamento,Nenhum,LO,OK,0,Usina em Obras em andamento
46828,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [180]:
slp = "Sem LP"
sim = "Sim"
tabela_criterio['criterio_novo'] = np.select(
    [
        (tabela_criterio.IdcSemPrevisao == sim),
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        (tabela_criterio.CondicaoAmbiental == "Sem LP"),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabela_criterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
8748,Em andamento,ACL,LI,OK,0,Usina em Obras em andamento
61220,Não Iniciada,Nenhum,LO,Verificar,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
53644,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"
1069,Não Iniciada,ACL,LI,Sem Uso,1,Sem obras e PPA Ok
2610,Em andamento,Ambos,LI,OK,0,Usina em Obras em andamento
36419,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
8177,Em andamento,ACL,LI,OK,0,Usina em Obras em andamento
55192,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
25908,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
34859,Não Iniciada,ACL,LO,OK,1,Sem obras e PPA Ok


In [181]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100


In [182]:
cols = ["DatObraigacaoOperacaoComercial","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'DatObraigacaoOperacaoComercial': 'OC_DatObraigacaoOperacaoComercial_mx','FaseAtual':'FASE','Indicador':'Ind_crono_norm', 'Dat_OC_DatObraigacaoOperacaoComercial':'DatObraigacaoOperacaoComercial'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)
calculo_previsao.drop(['SigTipoGeracao'],inplace=True,axis=1)

In [183]:
criterios_da_previsao  = pd.merge(
                                    tabela_criterio,calculo_previsao,
                                    on="IdeUsinaOutorga",how='inner'
                                    )

In [224]:
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regra_antiga'] = criterios_da_previsao[['Previsao_OC','DatObraigacaoOperacaoComercial']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regra_antiga'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','DatObraigacaoOperacaoComercial']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] >= 3) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','DatObraigacaoOperacaoComercial']].max(axis=1)
criterios_da_previsao.loc[criterios_da_previsao['criterio'] >= 3 & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','DatObraigacaoOperacaoComercial']].max(axis=1)



criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regrapmo'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8) ),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','DatObraigacaoOperacaoComercial']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','DatObraigacaoOperacaoComercial','Previsao_OC']].max(axis=1)


criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(3*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','DatObraigacaoOperacaoComercial','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(4*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','DatObraigacaoOperacaoComercial','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5) | (criterios_da_previsao['criterio'] == 7),'regrapmo'] =  criterios_da_previsao[['DatPrevistaComercial','DatObraigacaoOperacaoComercial','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'regrapmo'] = pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativapmo'] = "Analisar justificativa"
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & ( criterios_da_previsao['Previsao_OC'] > criterios_da_previsao['DatPrevistaComercial']),'dscjustificativapmo'] = "Estágio Atual das Obras"
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & ( criterios_da_previsao['Previsao_OC'] < criterios_da_previsao['DatPrevistaComercial']),'dscjustificativapmo'] = "Estágio Atual das Obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dscjustificativapmo'] = "Usina liberada para testes"
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dscjustificativapmo'] = "Testes não concluídos"


In [ ]:
calculo_previsao[calculo_previsao.Indicador>50]

In [ ]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [ ]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ["DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","IdeUsinaOutorga","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DthEnvio","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DthEnvio': 'Data_Envio','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI

In [ ]:
cols = ["Data_Envio","Comiss_Real","canteiroReal","prev_IO","ME_Real_conc_eol","Conc_Trans_REAL","CC_real","DesvRio_real","Ench_Real","IdeUsinaOutorga","ME_real","IO_real","IOTrans_Real"]
realcronogramarapeel[cols].rename(columns=rename_cols)

In [ ]:
#WORK.UG_Real n serve pra nd, inputug tbm n 
real_cronograma_rapeel = pd.merge(vcronograma,monitoramentoleilao,on=['IdeUsinaOutorga','Data_Envio'])